In [1]:
#import warnings
#warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
import numpy as np
import config as cfg
import csv
import time
import math
import random
import cv2
from datetime import datetime
from tensorflow.keras import backend as K

print('Python version : ', sys.version)
print('TensorFlow version : ', tf.__version__)
print('Keras version : ', keras.__version__)

Python version :  3.6.8 (default, Oct  7 2019, 12:59:55) 
[GCC 8.3.0]
TensorFlow version :  1.15.0
Keras version :  2.3.1


Using TensorFlow backend.


In [2]:
# 데이터 불러오기

xs = []
ys = []
num_classes = cfg.NUM_KEYS

#read data.csv
increased_path = '../dataset/20200512/3-increased'
with open(increased_path + '/data.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        # print(row[0], row[1])
        xs.append(increased_path + '/' + row[0])
        ys.append(int(row[1]))

        
increased_path = '../dataset/20200513/3-increased'
with open(increased_path + '/data.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        # print(row[0], row[1])
        xs.append(increased_path + '/' + row[0])
        ys.append(int(row[1]))
        
#read data.csv
increased_path = '../dataset/training-data-shortcut/3-increased'
with open(increased_path + '/data.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        # print(row[0], row[1])
        xs.append(increased_path + '/' + row[0])
        ys.append(int(row[1]))

        
increased_path = '../dataset/training-data-stop/3-increased'
with open(increased_path + '/data.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        # print(row[0], row[1])
        xs.append(increased_path + '/' + row[0])
        ys.append(int(row[1]))


# 파일명을 이용해서 실제 이미지 불러오기
num_images = len(xs)
assert len(xs) == len(ys)
x_images = []
y_labels = []

for i in range(num_images):
    filepath = xs[i]
    colorIMG = cv2.imread(filepath)[cfg.modelheight:]
    grayIMG = cv2.cvtColor(colorIMG, cv2.COLOR_BGR2GRAY)
    x_images.append(cv2.resize(grayIMG, dsize=(cfg.final_width, cfg.final_height)) / 255.0)
    y_labels.append([ys[i]])

#x_test_images = tf.reshape(x_test_images, [len(x_test_images), cfg.final_height, cfg.final_width, 1])
x_images = np.array(x_images)
x_images.reshape(x_images.shape[0], cfg.final_height, cfg.final_width, 1)
x_images = np.expand_dims(x_images, axis=-1)

y_labels = keras.utils.to_categorical(y_labels, num_classes)


print(len(x_images))  # 전체 데이터 수
print(len(y_labels))  # 전체 데이터 수
print()

print(x_images.shape)
print(y_labels.shape)
print()

print(x_images[0].shape)  # 학습용 이미지 한개의 차원
print(x_images[0].shape)  # 학습용 레이블 한개의 차원

print('data load ... done')

38483
38483

(38483, 66, 100, 1)
(38483, 6)

(66, 100, 1)
(66, 100, 1)
data load ... done


In [3]:
K.set_learning_phase(0)

# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('save/my_model.h5')
print('model ... loaded')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
model ... loaded


In [4]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 66, 100, 32)       832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 33, 50, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 33, 50, 64)        8256      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 25, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 25, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 25600)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)             

In [5]:
score = new_model.evaluate(x_images, y_labels, verbose=0)
#print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test accuracy: 0.9898397


In [6]:
correct_num = 0
missed_forward = 0
missed_left = 0
missed_right = 0
missed_upleft = 0
missed_upright = 0
missed_etc = 0

for i in range(num_images):
    filepath = xs[i]
    full_image = cv2.imread(filepath)
    full_image_gray = cv2.cvtColor(full_image, cv2.COLOR_BGR2GRAY)
    image_feed2model = cv2.resize(full_image_gray[cfg.modelheight:], dsize=(cfg.final_width, cfg.final_height)) / 255.0
    image_feed2model = np.array(image_feed2model)
    image_feed2model = np.expand_dims(image_feed2model, axis=-1)
    image_feed2model = np.expand_dims(image_feed2model, axis=0)
    
    #new_model.evaluate(image_feed2model)
    ret = new_model.predict(image_feed2model, batch_size=1)
    #print(ret)
    wheel = np.argmax(ret)
    #print(wheel)
    
    if int(ys[i]) == wheel:
        correct_num += 1
    else:
        if int(ys[i]) == cfg.LEFT:  # LEFT
            missed_left += 1
        elif int(ys[i]) == cfg.UP:  # FORWARD
            missed_forward += 1
        elif int(ys[i]) == cfg.RIGHT:  # RIGHT
            missed_right += 1
        elif int(ys[i]) == cfg.UP_LEFT:
            missed_upleft += 1
        elif int(ys[i]) == cfg.UP_RIGHT:
            missed_upright += 1
        else:  # etc
            missed_etc = missed_etc + 1
            
    if (i > 0) and (i % 1000 == 0):
        print('확인한 사진 수 :', i, '| 정확도: ', round((correct_num/i)*100, 2))
            
# summary
print('전체 이미지 수 ' + str(len(xs)))
print('LEFT 를 못맞춘 경우의 수: ' + str(missed_left))
print('RIGHT 를 못맞춘 경우의 수: ' + str(missed_right))
print('FORWARD 를 못맞춘 경우의 수: ' + str(missed_forward))
print('FORWARD-LEFT 를 못맞춘 경우의 수: ' + str(missed_upright))
print('FORWARD-RIGHT 를 못맞춘 경우의 수: ' + str(missed_upleft))
print('ETC 를 못맞춘 경우의 수: ' + str(missed_etc))

확인한 사진 수 : 1000 | 정확도:  97.7
확인한 사진 수 : 2000 | 정확도:  98.2
확인한 사진 수 : 3000 | 정확도:  98.2
확인한 사진 수 : 4000 | 정확도:  98.25
확인한 사진 수 : 5000 | 정확도:  98.22
확인한 사진 수 : 6000 | 정확도:  97.92
확인한 사진 수 : 7000 | 정확도:  97.89
확인한 사진 수 : 8000 | 정확도:  98.11
확인한 사진 수 : 9000 | 정확도:  98.21
확인한 사진 수 : 10000 | 정확도:  98.31
확인한 사진 수 : 11000 | 정확도:  98.41
확인한 사진 수 : 12000 | 정확도:  98.52
확인한 사진 수 : 13000 | 정확도:  98.59
확인한 사진 수 : 14000 | 정확도:  98.65
확인한 사진 수 : 15000 | 정확도:  98.67
확인한 사진 수 : 16000 | 정확도:  98.74
확인한 사진 수 : 17000 | 정확도:  98.8
확인한 사진 수 : 18000 | 정확도:  98.81
확인한 사진 수 : 19000 | 정확도:  98.72
확인한 사진 수 : 20000 | 정확도:  98.67
확인한 사진 수 : 21000 | 정확도:  98.58
확인한 사진 수 : 22000 | 정확도:  98.59
확인한 사진 수 : 23000 | 정확도:  98.61
확인한 사진 수 : 24000 | 정확도:  98.6
확인한 사진 수 : 25000 | 정확도:  98.65
확인한 사진 수 : 26000 | 정확도:  98.63
확인한 사진 수 : 27000 | 정확도:  98.63
확인한 사진 수 : 28000 | 정확도:  98.67
확인한 사진 수 : 29000 | 정확도:  98.68
확인한 사진 수 : 30000 | 정확도:  98.72
확인한 사진 수 : 31000 | 정확도:  98.74
확인한 사진 수 : 32000 | 정확도:  98.78
확인한 사진 수 : 33000 | 정확도